# SQuAD Question Answering

## 1 Introduction

Welcome to the "Enhancing Question Answering with Transformer Models" project! In this endeavor, I will be delving into the realm of Natural Language Processing to tackle the challenging task of building a model that can accurately comprehend and answer questions based on given contexts. By harnessing the transformative capabilities of Transformer architectures, I aim to create a robust system that not only understands the nuances of human language but also delivers contextually relevant answers.

The heart of our project beats with the transformative potential of Transformer architecture, a groundbreaking innovation that has revolutionized the field of NLP. Inspired by the "Attention is All You Need" paper, we will harness the capabilities of self-attention mechanisms, multi-head attention, and feedforward neural networks to build models that can efficiently capture intricate linguistic relationships, even in lengthy and complex text.

As I progress through this project, I'll explore data preprocessing, model selection, fine-tuning, and evaluation methodologies, aiming to equip the model with the ability to interpret context and contextually generate insightful answers. Whether it's tackling questions on passages of text, summarizing content, or generating human-like responses, this project is a tribute to the power of modern AI in understanding and manipulating language.

## 2 Project Setup

### 2.1 Packages Installation

In [1]:
try:
    import torch
    import torchvision
    from transformers import pipeline
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    !pip install torch torchvision
    !pip install transformers
    import torch
    import torchvision
    from transformers import pipeline
    print(f"torch version: {torch.__version__}")

torch version: 2.0.1
torchvision version: 0.15.2


In [2]:
classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598050713539124},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

### 2.2 Acquiring The SQuAD Dataset

## 3 Data Exploration

## 4 Data Preprocessing

## 5 Designing a Baseline

### 5.1 Model Selection

### 5.2 Model Finetuning

### 5.3 Training

### 5.4 Evaluation

## 6 Implementing Transformer-based Models

## 7 Using the Best Model

## 8 Conclusion